# Amazon Reviews - Model

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession, SQLContext
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("amazon-reviews-project").getOrCreate()

In [2]:
#for now, only reading reviews for items in the "Kitchen" category
reviews = sqlContext.read.parquet("s3://amazon-reviews-pds/parquet/product_category=Kitchen/")

***

## Data Extraction
Obtaining sentiment polarity from review string contents

In [3]:
reviews = reviews.na.fill({'review_body': '', 'review_headline': ''})

In [4]:
from pyspark.sql import Row
from pyspark.sql.functions import udf
from textblob import TextBlob

polarity = udf(lambda x: TextBlob(x).sentiment.polarity)

reviews = reviews.withColumn('headline_polarity', polarity('review_headline'))\
                 .withColumn('body_polarity', polarity('review_body'))

## Data Schema

In [5]:
reviews.count()

4882831

In [6]:
reviews.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = false)
 |-- review_body: string (nullable = false)
 |-- review_date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- headline_polarity: string (nullable = true)
 |-- body_polarity: string (nullable = true)



# Creating "helpful?" variable
##### A review is helpful if at least 75% of 'total_votes' have been 'helpful_votes'.

In [8]:
import pyspark.sql.functions as f
reviews = reviews.withColumn("helpful-ratio", reviews.helpful_votes/reviews.total_votes)

In [10]:
reviews = reviews.withColumn("helpful?", f.when(reviews["helpful-ratio"] > 0.75, 1).otherwise(0))

In [11]:
reviews.take(1)

[Row(marketplace='US', customer_id='17050420', review_id='R3QP7FJW6GAB3M', product_id='B007T0CIVS', product_parent='224029078', product_title='Preethi Eco Twin Jar Mixer Grinder, 550-Watt', star_rating=5, helpful_votes=3, total_votes=4, vine='N', verified_purchase='Y', review_headline='Finally something I can use', review_body="I always believed that the American blenders went toe to toe with the Indian ones, at least the affordable ones. I was getting tired of the blenders that only seemed to work when water was added or spice mixers that just tossed stuff around without actually grinding anything.<br /><br />Finally here's something that is pretty basic by Indian standards but so very effective. It comes with 2 jar and 4 lids. there are 2 lids for each jar. One adds a lot of room and the other one reduces it. I use the Large lid when I'm blending something like Dosa batter and the smaller lid for chutneys.<br /><br />There's also one extra blade (am not sure if this is for a special 

In [22]:
import pyspark.ml.evaluation as ev
from pyspark.ml import Pipeline
import pyspark.ml.regression as rg
import pyspark.sql.functions as f
import pyspark.ml.feature as feat
import pyspark.ml.classification as cl

In [24]:
# running bucketizer for pickup_longitude and adding it in the dataset
splits = [-float("inf"), 0, 5, float("inf")]

bucketizer = feat.Bucketizer(splits=splits, inputCol="year", outputCol="year_bkt")

reviews = bucketizer.transform(reviews)

In [27]:
reviews = reviews.drop('customer_id','review_id','product_id','parent_product','product title', 'helpful_votes', 'review_headline', 'review_body', 'review_date', 'year', 'helpful-ratio')

In [30]:
reviews.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- headline_polarity: float (nullable = true)
 |-- body_polarity: float (nullable = true)
 |-- helpful?: integer (nullable = false)
 |-- year_bkt: double (nullable = true)



In [29]:
from pyspark.sql.types import FloatType
reviews = reviews.withColumn("headline_polarity", reviews["headline_polarity"].cast(FloatType()))
reviews = reviews.withColumn("body_polarity", reviews["body_polarity"].cast(FloatType()))

In [32]:
reviews=reviews.drop('features') #removes the column 'features' if it already exists
#selects all numeric columns to be combined into column 'features'
Cols_to_Select = reviews["helpful?", "star_rating", "total_votes", "headline_polarity", "body_polarity", "year_bkt"]
assembler = feat.VectorAssembler(inputCols=Cols_to_Select.columns, outputCol="features") #creates the VectorAssembler object

In [33]:
# running the VectorAssembler transformation onto the dataframe to create the 'features' column
reviews=assembler.setHandleInvalid("skip").transform(reviews)

In [34]:
#splitting the data into train, test, and predict datasets
splitted_data = reviews.randomSplit([0.8, 0.2], 111)
train_data = splitted_data[0]
test_data = splitted_data[1]

In [ ]:
# creating the logistic regression object 
logReg_obj = cl.LogisticRegression(
    labelCol="helpful?"
    , featuresCol = "features",
    maxIter=5, regParam=0.3, elasticNetParam=0.8
)
# using pipeline to run the logistic regression, plus all other objects intially created
pipeline = Pipeline(
    stages=[
        logReg_obj
    ])

pipelineModel = pipeline.fit(train_data) #running the model on training dataset


In [ ]:
trainingSummary = pipelineModel.stages[-1].summary

print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

In [19]:
import pyspark.ml.evaluation as ev
#evaluating the model created against test dataset
results_logReg = (
    pipelineModel
    .transform(test_data)
    .select('helpful?', 'probability', 'prediction')
)

evaluator = ev.MulticlassClassificationEvaluator(
    predictionCol='prediction'
    , labelCol='helpful?')

(
    evaluator.evaluate(results_logReg)
    , evaluator.evaluate(
        results_logReg
        , {evaluator.metricName: 'weightedPrecision'}
    ) 
    , evaluator.evaluate(
        results_logReg
        , {evaluator.metricName: 'accuracy'}
    )
)

(0.9999997401694531, 0.9999997401695326, 0.9999997401694557)

In [ ]:
spark.stop()